# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Dive into the world of DeepLearning.AI and unlock the potential of deep learning in a seamless and engaging manner. With courses taught by industry experts like Andrew Ng, you can master the ins and outs of deep learning, neural networks, and beyond. Whether you're a beginner or an experienced professional, there's something for everyone. Gain cutting-edge skills, work on real-world projects, and propel your career to new heights. Join the DeepLearning.AI community today and revolutionize the way you perceive artificial intelligence. Elevate your knowledge, transform industries, and shape the future with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Dive into the world of DeepLearning.AI and unlock the potential of deep learning in a seamless and engaging manner. With courses taught by industry experts like Andrew Ng, you can master the ins and outs of deep learning, neural networks, and beyond. Whether you're a beginner or an experienced professional, there's something for everyone. Gain cutting-edge skills, work on real-world projects, and propel your career to new heights. Join the DeepLearning.AI community today and revolutionize the way you perceive artificial intelligence. Elevate your knowledge, transform industries, and shape the future with DeepLearning.AI.

-------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Dive into the world of DeepLearning.AI and unlock the potential of deep learning in a seamless and engaging manner. With courses taught by industry experts like Andrew Ng, you can master the ins and outs of deep learning, neural networks, and beyond. Whether you're a beginner or an experienced professional, there's something for everyone. Gain cutting-edge skills, work on real-world projects, and propel your career to new heights. Join the DeepLearning.AI community today and revolutionize the way you perceive artificial intelligence. Elevate your knowledge, transform industries, and shape the future with DeepLearning.AI.

-------------------------------------------


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated Feedback:
1. SEO Reviewer suggests optimizing content by targeting long-tail keywords, including internal links, and optimizing meta tags.
2. Legal Reviewer emphasizes ensuring compliance with advertisement regulations, checking for accuracy and truthfulness in claims, and avoiding misleading statements.

Final Suggestions:
1. Incorporate long-tail keywords and internal links to improve SEO.
2. Pay close attention to meta tags for better search engine visibility.
3. Ensure all claims are accurate and comply with advertisement regulations.
4. Avoid any misleading statements to maintain transparency and trust with the audience. 

Overall, it is essential to strike a balance between optimizing content for SEO purposes and ensuring legal compliance and accuracy in claims. By addressing both aspects effectively, the content can effectively reach its target audience while maintai

## Get the summary

In [15]:
print(res.summary)

Title: Revolutionize AI Learning with DeepLearning.AI

Unlock the world of deep learning with DeepLearning.AI, led by industry thought leader Andrew Ng. From neural networks to real-world projects, these courses cater to beginners and experts alike. Elevate your skills, propel your career, and shape the future of AI through practical learning. Join the DeepLearning.AI community today to stay at the forefront of artificial intelligence advancement.
